In [ ]:
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

sns.set()

dataset = pd.read_csv('../datasets/properati_argentina_2021.csv')


### Constantes

In [ ]:
SEMILLA = 0
TEST_SIZE = 0.2

### Funciones

In [ ]:
def separate_date(dataset, column_name):
  dataset[f'{column_name}_year'] = dataset[column_name].str[:4].astype(int)
  dataset[f'{column_name}_month'] = dataset[column_name].str[5:7].astype(int)
  dataset[f'{column_name}_day'] = dataset[column_name].str[8:10].astype(int)
  dataset.drop([column_name], axis=1, inplace=True)

# 1. Análisis Exploratorio y Preprocesamiento de Datos

### Separacion Train-Test

In [ ]:
dataset_train, dataset_test = train_test_split(dataset,
                                                test_size=TEST_SIZE,  
                                                random_state=SEMILLA) 

print("Proporcion - Train:", dataset_train.shape[0] / dataset.shape[0])
print("Proporcion - Test:", dataset_test.shape[0] / dataset.shape[0])

### Filtrado Set de Training

In [ ]:
dataset_train.columns.tolist()

In [ ]:
filtro_tipo_propiedad = ['casa', 'ph','departamento']
filtro_operacion = ['venta']
filtro_ubicacion = ['capital federal']
filtro_moneda = ['usd']

filtros = {
    'property_type': filtro_tipo_propiedad,
    'operation': filtro_operacion,
    'place_l2': filtro_ubicacion,
    'property_currency': filtro_moneda
}

In [ ]:
for columna in filtros:
  dataset_train[columna] = dataset_train[columna].str.lower()

In [ ]:
for columna_a_filtrar, filtro in filtros.items():
  dataset_train = dataset_train[dataset_train[columna_a_filtrar].isin(filtro)]

dataset_train.shape

In [ ]:
dataset_train.head()

Para este punto, tenemos el dataset filtrado por lo pedido e incluso normalizado (sin mayusculas)

In [ ]:
filas_totales = dataset_train.shape[0]
dataset_train.isna().sum()/filas_totales

Se puede apreciar que las columnas `place_l5` y `place_l6` contienen `NaN` en todas sus filas. Es por eso que nos deshacemos de ellas. `place_4` contiene `NaN` en el 96% de sus filas por lo que tambien la eliminamos.

Por el otro lado, el `id` es una variable unica e aleatoria para cada propiedad, por lo que no nos sirve para predecir nada. Por esta razon, la eliminamos tambien. Ocurre lo mismo con el titulo de la propiedad.

In [ ]:


columnas_eliminar = ['property_title', 'id', 'place_l4', 'place_l5', 'place_l6']
dataset_train.drop(columnas_eliminar, axis=1, inplace=True)
separate_date(dataset_train, 'start_date')
separate_date(dataset_train, 'end_date')
separate_date(dataset_train, 'created_on')

Tambien podemos sacar las columnas `property_currency`, `place_l2`, `operation` ya que estas solo contienen un valor posible (debido al filtrado)

In [ ]:
columnas_eliminar = ['property_currency', 'operation', 'place_l2']
dataset_train.drop(columnas_eliminar, axis=1, inplace=True)

In [ ]:
dataset_train.dtypes

-------------------------------------------------------------------

HASTA ACA LLEGA LA NORMALIZACION Y OBTENCION DE LOS DATOS DESEADOS

-------------------------------------------------------------------

In [ ]:
variables_cuantitativas = dataset_train.describe()
variables_cuantitativas

Dropeo la variable property_title ya que realmente no es una variable cualitativa. Si intentara obtener sus valores unicos, obtendria muchisimos distintos.

In [ ]:
variables_cualitativas = dataset_train.drop(variables_cuantitativas.columns, axis=1)
variables_cualitativas.nunique()

### Gráfico de las distribuciones de las variables cualitativas

In [ ]:


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 15), gridspec_kw={"width_ratios":[5, 3]})

frecuencia_absoluta = variables_cualitativas["place_l3"].value_counts()
frecuencia_relativa = frecuencia_absoluta / frecuencia_absoluta.sum()
sns.barplot(ax=axes[0], y=frecuencia_relativa.index, x=frecuencia_relativa.values).set_title("Distribucion de Barrios")

frecuencia_absoluta = variables_cualitativas["property_type"].value_counts()
frecuencia_relativa = frecuencia_absoluta / frecuencia_absoluta.sum()

sns.barplot(ax=axes[1], x=frecuencia_relativa.index, y=frecuencia_relativa.values).set_title("Distribucion del tipo de propiedad")
axes[1].tick_params('y', labelrotation=90)



### Correlaciones existentes entre las variables

In [ ]:


variables_cuantitativas = dataset_train[variables_cuantitativas.columns]
matriz_correlaciones = variables_cuantitativas.corr()
sns.reset_defaults()
sns.set(rc={"figure.figsize":(10, 10)})
sns.heatmap(data=matriz_correlaciones, annot=True, fmt=".2f")


Podemos ver que las variables que estan correlacionadas son: 
 - `property_rooms` con `property_bedrooms`
 - `property_surface_covered` con `property_surface_total`.
 - Con un grado mucho menor, `property_price` esta relacionado con `property_rooms` y `property_bedrooms`


In [ ]:
sns.reset_defaults()
sns.set()
sns.scatterplot(data=dataset_train, x="property_rooms", y="property_bedrooms")

In [ ]:
sns.reset_defaults()
sns.set()
sns.scatterplot(data=dataset_train, x="property_surface_covered", y="property_surface_total")


Podemos ver que la variables están muy relacionadas, excepto por un conjunto de puntos en el eje y. Esto se puede deber a los datos. Si graficamos las propiedades cuya superficie es mayor a 1000 (por proponer un numero arbitrario), vemos que el grafico tiene mucha correlacion positiva

In [ ]:
filtrado=dataset_train[(dataset_train.property_surface_covered > 1000) & (dataset_train.property_surface_total > 1000)]

sns.reset_defaults()
sns.set()
sns.scatterplot(data=filtrado, x="property_surface_covered", y="property_surface_total")



### Analisis de datos nulos

In [ ]:
filas_totales = dataset_train.shape[0]
dataset_train.isna().sum()/filas_totales

In [ ]:
distribucion_faltantes = (dataset_train.isna().sum()/filas_totales)
distribucion_faltantes = distribucion_faltantes[distribucion_faltantes.values != 0]
distribucion_faltantes = distribucion_faltantes.sort_values(ascending=False)

sns.set(rc={"figure.figsize":(16, 5)})
sns.barplot(x=distribucion_faltantes.index, y=distribucion_faltantes.values).set_title("Distribucion de Datos Faltantes")

In [ ]:
distribucion_faltantes_por_fila = dataset_train.isna().sum(axis=1).value_counts()
sns.barplot(x=distribucion_faltantes_por_fila.index, y=distribucion_faltantes_por_fila.values / dataset_train.shape[0]).set(xlabel="Cantidad de datos faltantes por fila", ylabel="proporcion", title="proporcion de datos faltantes por fila")

Vemos que las cantidad de filas con 4, 5, 6 datos faltantes son despreciables

### Imputacion de Datos Faltante

Algunas estrategias para imputar los datos faltantes:

- Sustitución de Casos: Se reemplaza con valores no observados. Hay que consultar con un experto.
- Sustitución por Media: Se reemplaza utilizando la medida calculada de los valores presentes. Sin embargo, esto trae consecuencias
    - La varianza estimada por la nueva variable no es válida ya que es atenuada por los valores repetidos
    - Se distorsiona la distribución
    - Las correlaciones que se observen estaran deprimidas debido a la repetición de un solo valores constante.
- Imputación Cold Deck: Se pueden obtener los datos faltantes a partir de otras variables del dataset.
- Imputacion Hot Deck: Se reemplazan los datos faltante con los valores que resultan más “similares”. Tenemos que definir que es “similar”, para ello se puede usar la tecnica: K vecinos más cercanos.
- Imputación por Regresión: El dato faltante es reemplaza con el valor predicho por un modelo de regresión.
- MICE (multivariate imputation by chained equations): Trabaja bajo el supuesto de que el origen de los datos es MAR. Es un proceso de imputación iterativo, donde cada iteración los valores faltantes se predicen en función de las variables restantes. El proceso se repite hasta que se encuentre consistencia en los datos (usualmente 10 iteraciones es suficiente). La primera iteración se realiza por uno de los métodos vistos anteriormente para rellenar los datos faltantes.

Anteriormente observamos que hay una correlacion entre `property_bedrooms`, `property_rooms` y `property_price` Podemos entrenar un modelo lineal para predecir los datos faltantes

Primero, graficamos la distribucion de las tres columnas

In [ ]:
columnas=['property_rooms','property_price','property_bedrooms']


sns.reset_defaults()
sns.set()

figs, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 8))

sns.histplot(ax=axes[0], data=dataset_train, x='property_rooms', discrete=True, stat="density")
sns.histplot(ax=axes[1], data=dataset_train, x='property_price', kde=True)
sns.histplot(ax=axes[2], data=dataset_train, x='property_bedrooms', discrete=True, stat="density")

plt.show()


In [ ]:
columnas_a_imputar = dataset_train[columnas].copy()

modelo_lineal = LinearRegression()
imp = IterativeImputer(estimator=modelo_lineal, missing_values=np.nan, max_iter=20, verbose=0, random_state=SEMILLA) 
columnas_imputadas = imp.fit_transform(columnas_a_imputar)

dataframe_imputado = pd.DataFrame(columnas_imputadas, columns=columnas).astype(int)

In [ ]:
sns.reset_defaults()
sns.set()

figs, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 8))

sns.histplot(ax=axes[0], data=dataframe_imputado, x='property_rooms', discrete=True, stat="density")
sns.histplot(ax=axes[1], data=dataframe_imputado, x='property_price', kde=True)
sns.histplot(ax=axes[2], data=dataframe_imputado, x='property_bedrooms', discrete=True, stat="density")

plt.show()

Observamos que el nuevo dataframe tiene distribucion similar a la que tenia antes de imputar, por lo que podemos sobreescribirlo en el dataframe de testing

In [ ]:
dataset_train[columnas] = dataframe_imputado[columnas].values


Como habiamos observado, habia una correlacion entre `property_surface_covered` y `property_surface_total`, por lo que podemos repetir el proceso anterior con estas variables

In [ ]:
columnas=['property_surface_covered','property_surface_total']


sns.reset_defaults()
sns.set()

figs, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))

sns.histplot(ax=axes[0], x=dataset_train.property_surface_covered, bins=100)
sns.histplot(ax=axes[1], x=dataset_train.property_surface_total, bins=100)

plt.show()
